In [1]:
# given a search query, return all results which are related that query

In [1]:
from datasets import load_dataset
import pandas as pd
from helpers import clean_up_tags

In [2]:
hf_datasets = load_dataset('nkasmanoff/huggingface-datasets')

/Users/noahkasmanoff/anaconda3/envs/nlp/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 488.73it/s]


Dataset parquet downloaded and prepared to /Users/noahkasmanoff/.cache/huggingface/datasets/nkasmanoff___parquet/nkasmanoff--huggingface-datasets-60bbbd3d2e18598e/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 300.73it/s]


In [10]:
hf_df = hf_datasets['train'].to_pandas()

In [11]:
hf_df['tags_cleaned'] = hf_df['tags'].apply(clean_up_tags)
hf_df.dropna(subset=['description'],inplace=True)
hf_df['description_full'] = hf_df['description'].fillna('') + ' ' + hf_df['tags_cleaned']
hf_df = hf_df[hf_df['description_full'] != ' ']
hf_df = hf_df[['id','description_full']]

In [12]:
hf_df.shape

(2804, 2)

In [11]:
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import DataFrameLoader
from langchain.llms import OpenAI


In [9]:
loader = DataFrameLoader(hf_df, page_content_column="description_full")
documents = loader.load()

In [12]:

# split the documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
# select which embeddings we want to use
embeddings = OpenAIEmbeddings()
# create the vectorestore to use as the index
db = Chroma.from_documents(texts, embeddings)
# expose this index in a retriever interface
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})
# create a chain to answer questions 


Created a chunk of size 1253, which is longer than the specified 1000
Created a chunk of size 1253, which is longer than the specified 1000
Created a chunk of size 1253, which is longer than the specified 1000
Created a chunk of size 1253, which is longer than the specified 1000
Created a chunk of size 1253, which is longer than the specified 1000
Created a chunk of size 1253, which is longer than the specified 1000
Created a chunk of size 1140, which is longer than the specified 1000
Created a chunk of size 1140, which is longer than the specified 1000
Created a chunk of size 1087, which is longer than the specified 1000
Created a chunk of size 1190, which is longer than the specified 1000
Created a chunk of size 1164, which is longer than the specified 1000
Created a chunk of size 1190, which is longer than the specified 1000
Created a chunk of size 1242, which is longer than the specified 1000
Created a chunk of size 1187, which is longer than the specified 1000
Created a chunk of s

In [46]:
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=True)
query = "Show me datasets which tell me about translation?"
result = qa({"query": query})
reponse_text = result['result']
source_documents = result['source_documents']
linked_datasets = [x.metadata['id'] for x in source_documents]

print("Answer: ", reponse_text)
print("Related Datasets: ", linked_datasets)


Answer:   The new dataset mentioned in the context is a massively multilingual dataset derived from TED Talk transcripts which consists of parallel arrays of language and text.
Related Datasets:  ['frtna/jwt300_mt', 'ted_multi']


['SharedBailii/NER-BAILII-UK-CCA', 'svnfs/depth-of-field']